In [373]:
import pandas as pd
train = pd.read_csv("titanic/train.csv")
test = pd.read_csv("titanic/test.csv")
print(len(train),len(test))

891 418


In [374]:
full = pd.concat([train.drop("Survived",axis=1),test])
print(len(full))

1309


In [375]:
full.info()
full.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [376]:
full.describe(include=['O'])#Object

,Name,Sex,Ticket,Cabin,Embarked
count,1309,1309,1309,295,1307
unique,1307,2,929,186,3
top,"Connolly, Miss. Kate",male,CA. 2343,C23 C25 C27,S
freq,2,843,11,6,914


In [377]:
full.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [378]:
full.Age = full.Age.fillna(full.Age.median())
del full["Cabin"]
full.Fare = full.Fare.fillna(full.Fare.median())
full.Embarked = full.Embarked.fillna(full.Embarked.mode()[0])
full.isnull().sum()
del full["PassengerId"]

In [379]:
full["p_1"] = pd.get_dummies(full.Pclass)[1]
full["p_2"] = pd.get_dummies(full.Pclass)[2]
full["p_3"] = pd.get_dummies(full.Pclass)[3]
del full["Pclass"]

In [380]:
full["family_size"] = full["SibSp"]+full["Parch"]+1
del full["SibSp"],full["Parch"]

In [381]:
full["Em_S"] = pd.get_dummies(full.Embarked)["S"]
full["Em_C"] = pd.get_dummies(full.Embarked)["C"]
full["Em_Q"] = pd.get_dummies(full.Embarked)["Q"]
del full["Embarked"]

In [382]:
full.Sex = full.Sex.replace({"male":0,"female":1})
full = full.drop("Ticket",axis=1)

In [383]:
for i in range(len(full)):
    if "Mr" in full.iloc[i,0]:
        full.iloc[i,0]="Mr"
    elif "Miss" in full.iloc[i,0]:
        full.iloc[i,0]="Miss"
    elif "Master" in full.iloc[i,0]:
        full.iloc[i,0]="Master"
    elif "Dr" in full.iloc[i,0]:
        full.iloc[i,0]="Dr"
    else:
        full.iloc[i,0]="unique"

In [384]:
full["Mr"] = pd.get_dummies(full.Name)["Mr"]
full["Miss"] = pd.get_dummies(full.Name)["Miss"]
full["Master"] = pd.get_dummies(full.Name)["Master"]
full["Dr"] = pd.get_dummies(full.Name)["Dr"]
full["unique"] = pd.get_dummies(full.Name)["unique"]
del full["Name"]

In [385]:
for i in range(len(full)):
    if full.iloc[i,1]<4:
        full.iloc[i,1]=0
    elif full.iloc[i,1]<15:
        full.iloc[i,1]=1
    elif full.iloc[i,1]<25:
        full.iloc[i,1]=2
    elif full.iloc[i,1]<40:
        full.iloc[i,1]=3
    else:
        full.iloc[i,1]=4
full["Age_infants"] = pd.get_dummies(full.Age)[0]
full["Age_young"] = pd.get_dummies(full.Age)[1]
full["Age_adult"] = pd.get_dummies(full.Age)[2]
full["Age_middle"] = pd.get_dummies(full.Age)[3]
full["Age_old"] = pd.get_dummies(full.Age)[4]
del full["Age"]

In [386]:
from sklearn.model_selection import train_test_split
X = full.iloc[:891,:]
y = train.Survived
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size = 0.3,random_state=0)

In [405]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
for i in range(2,7):
    for j in range(3,11,2):
        for k in [100,200,300]:
            clf = xgb.XGBClassifier(max_depth=i, n_estimators=k, learning_rate=0.05)
            model = cross_validate(clf,train_X,train_y,cv=j,return_train_score=True)
            clf.fit(train_X.iloc[:(int(819/j*(j-1)))],train_y[:(int(819/j*(j-1)))])
            s = clf.score(test_X,test_y)
            a = model["test_score"].std()
            if s>0.83 and a<0.02:
                print("i:",i,"j:",j,"k:",k)
                print(model["train_score"].mean(),model["train_score"].std())
                print(model["test_score"].mean(),model["test_score"].std())
                clf.fit(train_X.iloc[:(int(819/j*(j-1)))],train_y[:(int(819/j*(j-1)))])
                print("testscore:",clf.score(test_X,test_y))
                pred1=clf.predict(full.iloc[891:])
                print(418-(pred1==pred).sum())

i: 3 j: 3 k: 100
0.8547381835032438 0.005596845724190234
0.8250495478756349 0.005712083533402414
testscore: 0.8470149253731343
14
i: 3 j: 3 k: 200
0.8763998300895891 0.01157509665906312
0.810610987241422 0.012367962587805887
testscore: 0.8507462686567164
26
i: 4 j: 3 k: 100
0.8780101173926474 0.0029821668884305276
0.80900068128329 0.014707981201564647
testscore: 0.8432835820895522
20
i: 4 j: 3 k: 300
0.910914813098548 0.00010103085767646457
0.8041929889755978 0.014611500816361167
testscore: 0.835820895522388
37
i: 6 j: 3 k: 100
0.9028904077849861 0.002217972553298737
0.8089929394277221 0.015809225448886927
testscore: 0.8544776119402985
23
i: 6 j: 5 k: 300
0.9261638644188575 0.007962091147986721
0.7993751152073733 0.01659182620217637
testscore: 0.832089552238806
29


In [395]:
full

,Sex,Fare,p_1,p_2,p_3,family_size,Em_S,Em_C,Em_Q,Mr,Miss,Master,Dr,unique,Age_infants,Age_young,Age_adult,Age_middle,Age_old
0,0,7.2500,0,0,1,2,1,0,0,1,0,0,0,0,0,0,1,0,0
1,1,71.2833,1,0,0,2,0,1,0,1,0,0,0,0,0,0,0,1,0
2,1,7.9250,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,0
3,1,53.1000,1,0,0,2,1,0,0,1,0,0,0,0,0,0,0,1,0
4,0,8.0500,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,1,0
5,0,8.4583,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,0
6,0,51.8625,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1
7,0,21.0750,0,0,1,5,1,0,0,0,0,1,0,0,1,0,0,0,0
8,1,11.1333,0,0,1,3,1,0,0,1,0,0,0,0,0,0,0,1,0
9,1,30.0708,0,1,0,2,0,1,0,1,0,0,0,0,0,1,0,0,0


In [406]:
clf = xgb.XGBClassifier(max_depth=3, n_estimators=100, learning_rate=0.05)
model = cross_validate(clf,train_X,train_y,cv=3,return_train_score=True)
print(model["train_score"].mean(),model["train_score"].std())
print(model["test_score"].mean(),model["test_score"].std())
clf.fit(train_X.iloc[:(int(819/3*2))],train_y[:(int(819/3*2))])
print("testscore:",clf.score(test_X,test_y))
pred=clf.predict(full.iloc[891:])
#print(418-(pred1==pred).sum())

0.8547381835032438 0.005596845724190234
0.8250495478756349 0.005712083533402414
testscore: 0.8470149253731343


In [407]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": pred
    })
submission.to_csv('submission.csv', index=False)